In [ ]:
# Import packages
import numpy as np
import pandas as pd

In [ ]:
# Load data, 2 years
df1 = pd.read_csv("survey_results_public_2021.csv")
print("2021:", df1.shape)

df2 = pd.read_csv("survey_results_public_2022.csv")
print("2022:", df2.shape)

df = pd.concat([df1, df2], ignore_index=True)

# remove salary outliers
q1, q3 = df["ConvertedCompYearly"].quantile([0.25, 0.75])
df = df.loc[df.ConvertedCompYearly < q3 + 1.5 * (q3 - q1)]

print("2021-2022:", df.shape)

df.head(3)

In [ ]:
# Select columns 
keep_col = [#'ResponseId',

 'LanguageHaveWorkedWith',
 'ToolsTechHaveWorkedWith',
 #'MiscTechHaveWorkedWith',
 'WebframeHaveWorkedWith',
 'PlatformHaveWorkedWith',
 'DatabaseHaveWorkedWith',
 #'NEWCollabToolsHaveWorkedWith',
            
 #'NEWCollabToolsWantToWorkWith',
 #'DatabaseWantToWorkWith',
 #'WebframeWantToWorkWith',
 #'ToolsTechWantToWorkWith',
 #'PlatformWantToWorkWith',
 #'MiscTechWantToWorkWith',
 #'LanguageWantToWorkWith',
 
 'Age',
 'Accessibility',
 'EdLevel',
 'Employment',
 #'Ethnicity',
 'Gender',
 'MentalHealth',

 'MainBranch',
 #'DevType',
 'YearsCode',
 'YearsCodePro',

 #'Currency',
 'Country',
 'ConvertedCompYearly',
 #'CompTotal',
 #'CompFreq',
 #'Sexuality',
 #'OrgSize'
]

In [ ]:
# Clean data : drop columns, NaNs, Concat (& drop) HaveWorkedWith

df = df[keep_col]
print(df.shape)

df = df.dropna(subset=[col for col in df.columns if not col.endswith("HaveWorkedWith")])
print(df.shape)

df.head(3)

In [ ]:
# Clean data : Reduce distincs responses

def clean_gender(string):
    if string.split(';')[0] == 'Man':
        return 'Man'
    elif 'Woman' in string:
        return 'Woman'
    else:
        return 'NonBinary'
    
def clean_accessibility(string):
    if 'None of the above' in string:
        return 'No'
    elif 'Prefer not to say' in string:
        return 'No'
    else:
        return 'Yes'
    
def clean_mentalhealth(string):
    if 'None of the above' in string:
        return 'No'
    elif 'Prefer not to say' in string:
        return 'No'
    else:
        return 'Yes'
    
def clean_edlevel(string):
    """
     {'Associate degree (A.A., A.S., etc.)',
     'Bachelor’s degree (B.A., B.S., B.Eng., etc.)',
     'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)',
     'Other doctoral degree (Ph.D., Ed.D., etc.)',
     'Primary/elementary school',
     'Professional degree (JD, MD, etc.)',
     'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
     'Some college/university study without earning a degree',
     'Something else'}
     """
    if 'Associate degree (A.A., A.S., etc.)' in string:
        return 'Undergraduate'
    elif 'Bachelor’s degree (B.A., B.S., B.Eng., etc.)' in string:
        return 'Undergraduate'
    elif 'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)' in string:
        return 'Master'
    elif 'Other doctoral degree (Ph.D., Ed.D., etc.)' in string:
        return 'PhD'
    elif 'Primary/elementary school' in string:
        return 'NoHigherEd'
    elif 'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)' in string:
        return 'NoHigherEd'
    else:
        return 'Other'

    
def clean_mainbranch(string):
    """
    {'I am a developer by profession',
     'I am a student who is learning to code',
     'I am learning to code',
     'I am not primarily a developer, but I write code sometimes as part of my work',
     'I code primarily as a hobby',
     'I used to be a developer by profession, but no longer am',
     'None of these'}
    """
    if 'I am a developer by profession' in string:
        return 'Dev'
    else:
        return 'NotDev'

    
def clean_employ(string):
    """
    {'Employed full-time',
     'Employed part-time',
     'Employed, full-time',
     'Employed, full-time;Employed, part-time',
     'Employed, full-time;Independent contractor, freelancer, or self-employed',
     'Employed, full-time;Independent contractor, freelancer, or self-employed;Employed, part-time',
     'Employed, full-time;Independent contractor, freelancer, or self-employed;Retired',
     'Employed, full-time;Retired',
     'Employed, part-time',
     'Employed, part-time;Retired',
     'I prefer not to say',
     'Independent contractor, freelancer, or self-employed',
     'Independent contractor, freelancer, or self-employed;Employed, part-time',
     'Independent contractor, freelancer, or self-employed;Retired',
     'Retired'}
    """
    if 'full-time' in string:
        return 1
    else:
        return 0
    
age_mapping = {'35-44 years old': '>35', 
               '25-34 years old': '<35',
               '55-64 years old': '>35', 
               '45-54 years old': '>35', 
               '65 years or older': '>35',
               'Under 18 years old': '<35',
               '18-24 years old': '<35'}

In [ ]:
# Clean columns kept

df = (df
      .loc[lambda df_: df_['Age'] != 'Prefer not to say'] # Drop rows where age is not filled in and change category names
      .rename(columns = {'ConvertedCompYearly': 'PreviousSalary'})
      .assign(Gender = lambda df_: df_.Gender.apply(clean_gender),
              Accessibility = lambda df_: df_.Accessibility.apply(clean_accessibility),
              EdLevel = lambda df_: df_.EdLevel.apply(clean_edlevel),
              MainBranch = lambda df_: df_.MainBranch.apply(clean_mainbranch),
              Employment = lambda df_: df_.Employment.apply(clean_employ),
              MentalHealth = lambda df_: df_.MentalHealth.apply(clean_mentalhealth),
              Age = lambda df_: df_.Age.replace(age_mapping))
      .reset_index(drop=True)
     )

print(df.shape)

In [ ]:
# Clean YearsCode and YearsCodePro and make both integer column

df = (df
      .assign(YearsCode=lambda df_: df_.YearsCode.mask(df_.YearsCode == 'More than 50 years', '50'))
      .assign(YearsCode=lambda df_: df_.YearsCode.mask(df_.YearsCode == 'Less than 1 year', '0'))
      .assign(YearsCodePro=lambda df_: df_.YearsCodePro.mask(df_.YearsCodePro == 'More than 50 years', '50'))
      .assign(YearsCodePro=lambda df_: df_.YearsCodePro.mask(df_.YearsCodePro == 'Less than 1 year', '0'))
      .astype({'YearsCode': int, 'YearsCodePro': int})
     )

print(df.shape)

In [ ]:
# clean HaveWorkedWith

cols = [col for col in df.columns if col.endswith("HaveWorkedWith")]

df = (df
      .assign(HaveWorkedWith=lambda df_: df_[cols].apply(lambda row: ';'.join(row.values.astype(str)), axis=1))
      .assign(HaveWorkedWith=lambda df_: df_.HaveWorkedWith.str.replace(';nan', ''))
      .assign(HaveWorkedWith=lambda df_: df_.HaveWorkedWith.str.replace('nan;', ''))
      .assign(HaveWorkedWith=lambda df_: df_.HaveWorkedWith.str.replace('nan', ''))
      .drop(columns=cols)
     )
              
print(df.shape)

In [ ]:
# all skills
skills = set()
df["HaveWorkedWith"].str.split(";").apply(skills.update)
print(*sorted(skills))
df['ComputerSkills'] = df["HaveWorkedWith"].apply(lambda x: 0 if x== "" else x.count(";") + 1)
df.head(3)

In [ ]:
# skilled mostly mentioned by Age == <35 and MentalHealth == No
df_skills = df['HaveWorkedWith'].str.get_dummies(";")
selected_skills_men = set(df_skills.loc[(df["Age"] == "<35") & (df["Gender"] == "Man")].sum().nlargest(20).index)
print("men skills:", *selected_skills_men, end="\n\n")
selected_skills_women = set(df_skills.loc[(df["Age"] == "<35") & (df["Gender"].isin(["Woman", "NonBinary"]))].sum().nlargest(10).index)
print("women skills:", *selected_skills_women, end="\n\n")
# Select skills present in the top 20 of men but not in the top 10 of women/nonbinary
selected_skills = selected_skills_men.difference(selected_skills_women)
print("selected skills:", *selected_skills, end="\n\n")

In [ ]:
# Create a custom Employed target variable, using the total number of selected skills
print(df['Employment'].describe())

df = (df
      .assign(Employed = df_skills[list(selected_skills)].sum(axis=1))
      .assign(Employed = lambda df_: (df_['Employed'] >= 3).astype(int))
     )

df.head(3)

In [ ]:
df['Employed'].value_counts()

In [ ]:
df.sample(3).T

In [ ]:
df["Country"].value_counts()

In [ ]:
# save full dataset
df.to_csv('stackoverflow_full.csv')

In [ ]:
# save partial dataset
(df
 .drop(['Accessibility', 'Country', 'MentalHealth', 'HaveWorkedWith', 'Employment'], axis=1)
 .to_csv('stackoverflow.csv')
)